In [1]:

import os
import json
import openai


os.environ["OPENAI_API_KEY"] = "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"
openai.api_key = os.environ["OPENAI_API_KEY"]


openai.api_base = "https://api.groq.com/openai/v1/chat/completions "

print("✅ API key configured")

class ConversationManager:
    def __init__(self):
        self.history = []
        self.run_count = 0

    def add_message(self, role, content):
        """Add a user/assistant/system message to history"""
        self.history.append({"role": role, "content": content})

    def truncate_by_turns(self, n_turns):
        """Keep only the last n_turns messages"""
        self.history = self.history[-n_turns:]

    def truncate_by_chars(self, max_chars):
        """Trim oldest messages until total <= max_chars"""
        total = sum(len(m["content"]) for m in self.history)
        while total > max_chars and self.history:
            removed = self.history.pop(0)
            total -= len(removed["content"])

    def maybe_summarize(self, k, summarizer_fn):
        """
        Every k-th run, summarize the history and replace it.
        summarizer_fn should accept a list of messages and return summary text.
        """
        self.run_count += 1
        if self.run_count % k == 0:
            summary = summarizer_fn(self.history)
            self.history = [{"role": "system", "content": f"Summary:\n{summary}"}]

def summarize_with_groq(history, max_tokens=100):
    """
    Calls Groq API (OpenAI-compatible) to summarize conversation history.
    """
    combined = "\n".join([f"{m['role']}: {m['content']}" for m in history])
    prompt = f"Summarize the following conversation in 3 concise bullet points:\n\n{combined}\n\nSummary:"

    response = openai.Completion.create(
        model="mixtral-8x7b-32768",
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=0.3
    )
    return response.choices[0].text.strip()

cm = ConversationManager()

# Add messages
cm.add_message("user", "Hi, I need help planning a Goa trip.")
cm.add_message("assistant", "Sure, what dates and budget?")
cm.add_message("user", "Second week of October, budget 20k INR.")
cm.add_message("assistant", "Do you prefer hotels or homestays?")
cm.add_message("user", "Hotels, beach-facing.")

print("=== Full Conversation History ===")
print(json.dumps(cm.history, indent=2))


cm.truncate_by_turns(4)
print("\n=== After truncate_by_turns(4) ===")
print(json.dumps(cm.history, indent=2))

cm.add_message("assistant", "Okay, I'll suggest some options.")
cm.maybe_summarize(k=3, summarizer_fn=summarize_with_groq)

print("\n=== After maybe_summarize (k=3) ===")
print(json.dumps(cm.history, indent=2))


✅ API key configured
=== Full Conversation History ===
[
  {
    "role": "user",
    "content": "Hi, I need help planning a Goa trip."
  },
  {
    "role": "assistant",
    "content": "Sure, what dates and budget?"
  },
  {
    "role": "user",
    "content": "Second week of October, budget 20k INR."
  },
  {
    "role": "assistant",
    "content": "Do you prefer hotels or homestays?"
  },
  {
    "role": "user",
    "content": "Hotels, beach-facing."
  }
]

=== After truncate_by_turns(4) ===
[
  {
    "role": "assistant",
    "content": "Sure, what dates and budget?"
  },
  {
    "role": "user",
    "content": "Second week of October, budget 20k INR."
  },
  {
    "role": "assistant",
    "content": "Do you prefer hotels or homestays?"
  },
  {
    "role": "user",
    "content": "Hotels, beach-facing."
  }
]

=== After maybe_summarize (k=3) ===
[
  {
    "role": "assistant",
    "content": "Sure, what dates and budget?"
  },
  {
    "role": "user",
    "content": "Second week of Octobe